# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [1]:
import os
from glob import glob

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Retrieve the PRICE_DATA environment variable
price_data_path = os.getenv("PRICE_DATA")

# Use glob to get all parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive=True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

Let's first create a progress bar decorator

In [2]:
from dask.diagnostics import ProgressBar
from functools import wraps

def dask_progress_bar(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        with ProgressBar():
            return func(*args, **kwargs)
    return wrapper


In [3]:
import dask.dataframe as dd

# Load parquet files into Dask dataframe
dd_px = dd.read_parquet(parquet_files).reset_index()

# Add lags for Close and Adj Close
dd_px['Close_lag_1'] = dd_px.groupby('Ticker')['Close'].shift(1)
dd_px['Adj_Close_lag_1'] = dd_px.groupby('Ticker')['Adj Close'].shift(1)

# Calculate returns based on 'Close'
dd_px['returns'] = (dd_px['Close'] / dd_px['Close_lag_1']) - 1

# Add the high-low range for the day
dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

dd_feat = dd_px



C:\Users\AC\AppData\Local\Temp\ipykernel_12096\1423507564.py:7: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px['Close_lag_1'] = dd_px.groupby('Ticker')['Close'].shift(1)
C:\Users\AC\AppData\Local\Temp\ipykernel_12096\1423507564.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_px['Adj_Close_lag_1'] = dd_px.groupby('Ticker')['Adj Close'].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [4]:
# Apply the progress bar decorator to the compute function
@dask_progress_bar
def compute_dd_feat(dd_feat):
    return dd_feat.compute()

# Convert the Dask DataFrame to a Pandas DataFrame with the progress bar
dd_feat_pd = compute_dd_feat(dd_feat)

# Now calculate the moving average on the Pandas DataFrame
dd_feat_pd['moving_avg_10'] = dd_feat_pd['returns'].rolling(10).mean()


[########################################] | 100% Completed | 85m 58s


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it wasn't necessary to convert to pandas to calculate the moving average return. You can calculate the moving average directly in Dask using rolling(window).mean() without converting the DataFrame to pandas. Dask supports similar functionality for large datasets and can perform computations in parallel, making it more suitable for larger datasets that might not fit in memory when converted to pandas.

+ Would it have been better to do it in Dask? Why?

Yes, it would have been better to do it in Dask. Dask is optimized for handling large datasets by partitioning them and processing in parallel. It allows for out-of-core computations, meaning it doesn't need the entire dataset in memory. So if you were working with a very large dataset, Dask would be better equipped to handle this efficiently compared to pandas, which requires the full dataset to fit into memory.

(1 pt)

In [7]:
dd_feat['Moving_Avg_Return'] = dd_feat['returns'].rolling(10, min_periods=1).mean()


In [8]:
# Simulate Dask DataFrame
dd_feat = dd.from_pandas(dd_feat_pd, npartitions=12)

# Apply the progress bar decorator to the rolling average function
@dask_progress_bar
def calculate_rolling_average(df, window=10):
    df['moving_avg_10'] = df['returns'].rolling(window, min_periods=1).mean()
    return df

# Apply the function across all partitions using map_partitions
dd_feat_rolling = dd_feat.map_partitions(calculate_rolling_average)

# Trigger the computation (this will show the progress bar)
dd_feat_rolling = dd_feat_rolling.compute()

# Now dd_feat_rolling contains the result with the moving average of returns


In [9]:
# Show the head and tail of the Dask DataFrame after the computation
dd_feat_rolling_head_tail = dd_feat_rolling.head(5), dd_feat_rolling.tail(5)
dd_feat_rolling_head_tail


(Price Ticker       Date  Adj Close      Close       High        Low  \
 0         DD 2004-01-02        NaN  31.454390  31.928538  31.377914   
 0         DD 2003-01-02        NaN  22.360483  22.448574  21.663094   
 0        AEE 2006-01-03        NaN  23.784660  23.830523  23.495722   
 0        DFS 2006-01-03        NaN        NaN        NaN        NaN   
 0        DFS 2003-01-02        NaN        NaN        NaN        NaN   
 
 Price       Open     Volume  Year  Close_lag_1  Adj_Close_lag_1    returns  \
 0      31.790880  1581218.0  2004    44.836933              NaN   0.362712   
 0      21.986095  1636022.0  2003    15.702373              NaN   3.882691   
 0      23.711279   900500.0  2006     9.216395              NaN  -0.181300   
 0            NaN        NaN  2006    15.481647              NaN  22.015234   
 0            NaN        NaN  2003    41.166416              NaN   3.577728   
 
 Price  hi_lo_range  moving_avg_10  
 0         1.850461       0.362712  
 0         1.269

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.